In [1]:
import zipfile
import requests
from collections import Counter
import re
import math
import random

r = requests.get("https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
open("spam.zip", "wb").write(r.content)
zipfile.ZipFile("spam.zip").extractall("./")

In [7]:
# SMS를 Bag-of_Words로 분리
data = [line.strip().split("\t") for line in open("SMSSpamCollection.txt", "r")]
for d in data:
    d[1] = set(re.findall("[a-z0-1_]+", d[1].lower()))

# train, test 데이터 분리 
random.shuffle(data)
train_len = int(len(data) * 0.8)
test_len = len(data) - train_len

train = data[:train_len]
test = data[train_len:]

In [8]:
train_spam = [bow for cls, bow in train if cls == 'spam']
train_ham = [bow for cls, bow in train if cls == 'ham']

prior_spam = math.log(len(train_spam) / len(train))
prior_ham = math.log(len(train_ham) / len(train))

In [11]:
spam_words = Counter(word for bow in train_spam for word in bow)
ham_words = Counter(word for bow in train_ham for word in bow)


In [12]:
def predict(bow, spam_words, ham_words, n_spams, n_hams, alpha):
    spam_score = prior_spam + sum( math.log((spam_words[word] + alpha)/(n_spams + alpha)) for word in bow) 
    ham_score = prior_ham + sum( math.log((ham_words[word] + alpha)/(n_hams + alpha)) for word in bow)

    if spam_score < ham_score:
        return 'ham'
    else:
        return 'spam'

In [13]:
tp, tn, fp, fn = 0, 0, 0, 0
for cls, bow in test:
    pred = predict(bow, spam_words, ham_words, len(train_spam), len(train_ham), 1e-4)
    if cls == 'spam' and pred == 'spam': tp += 1
    elif cls == 'ham' and pred == 'ham': tn += 1
    elif cls == 'ham' and pred == 'spam': fp += 1
    else: fn += 1

print(f"tp {tp}, tn: {tn}, fp: {fp}, fn: {fn}")

tp 153, tn: 936, fp: 21, fn: 5


In [15]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)

print(f"accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {f1}")

accuracy: 0.9766816143497757, precision: 0.8793103448275862, recall: 0.9683544303797469, f1: 0.9216867469879519
